## Using a benchmark problem
This tutorial shows how to use and query a benchmark problem.

In [1]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry

**ry-c++-log** /home/jay/git/optimization-course/rai/rai/ry/ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback



**There a 6 benchmarks in total:** 3 scenarios, each with sequence or path variant. The sequence variants have dense Jacobians and optimize over a discrete sequence of few (maybe one) configurations; the path variants optimize over a whole continuous path and have sparse Jacobians.

*All problems neglect collisions.* Esp. the last one has lots of collisions. But there are other difficulties in the last one.

The following loads a benchmark problem, from which we get the ry.MathematicalProgram interface

In [2]:
#benchmark = ry.OptBench_Skeleton_Pick(ry.arg.path)  #1. scenario: simple pick, just one configuration or path
benchmark = ry.OptBench_Skeleton_Handover(ry.arg.path) #2. scenario: pick-handover-touch, 3 configurations or path
#benchmark = ry.OptBench_Skeleton_StackAndBalance(ry.arg.sequence) #3. scenario: pick boxes and stack
nlp = benchmark.get()

In [3]:
# If you want, get a full API:
#help(nlp)

The first method to use is `getInitializationSample`, which returns an initial x

In [5]:
x = nlp.getInitializationSample()

The most important method is `evaluate(x)`, which returns a tuple `(phi,J)`

In [6]:
(phi,J) = nlp.evaluate(x)
print('phi=', phi, '\nJ=', J)

phi= [-1.54400259e-01 -2.86335904e-01  8.35220444e-02 ... -2.15065688e+01
 -1.10033525e+02 -1.31540751e+02] 
J= [[ 0.00000000e+00  0.00000000e+00  1.46969385e+01]
 [ 1.00000000e+00  1.00000000e+00  1.46969385e+01]
 [ 2.00000000e+00  2.00000000e+00  1.46969385e+01]
 ...
 [ 1.76300000e+03  1.67300000e+03  1.30824359e+02]
 [ 1.76300000e+03  1.67300000e+03 -6.50401406e-01]
 [ 1.76300000e+03  1.67300000e+03  0.00000000e+00]]


Note the dimensionalities:

In [7]:
print('dim(x)=', nlp.getDimension())
print('dim(x)=', x.shape)
print('dim(phi)=', phi.shape)
print('dim(J)=', J.shape)

dim(x)= 1687
dim(x)= (1687,)
dim(phi)= (1764,)
dim(J)= (10980, 3)


The features `phi` define all cost terms and inequality/equality constraint values. The feature types tell you, which feature entries refer to costs/constraints. For each feature, an integer is returned -- but actually it is an enum. So you can test equality with ry.OT.f, ry.OT.sos, ry.OT.ineq, or ry.OT.eq

In [8]:
types = nlp.getFeatureTypes()
print(types)
print(types[0] == ry.OT.f)
print(types[0] == ry.OT.sos)
print(types[0] == ry.OT.ineq)
print(types[0] == ry.OT.eq)

[2 2 2 ... 4 4 4]
False
True
False
False


In the benchmarks, most features are features are sum-of-square features (e.g. from control costs). Some are equality constraints.

Finally, the benchmark problems implement a report method, which should help you to get more insight in what $x$ means and what is computed. For high verbosity it also displays something - in our case the robot in pose $x$.

**Hit ENTER in the displayed window** to continue.

In [10]:
# the report method generally returns some info string - internal information given by the benchmark implementation
print(nlp.report(4))

KOMO Problem:
  x-dim:1687  dual-dim:0
  T:90 k:2 phases:3 stepsPerPhase:30 tau:0.166667
  #timeSlices:92 #totalDOFs:1687 #frames:5152  #pathQueries:1
    times:[0.166667, 0.333333, 0.5, 0.666667, 0.833333, 1, 1.16667, 1.33333, 1.5, 1.66667]
  computeCollisions:0
    TASK 'F_qItself/2-#28' (-2..89)  type:sos  order:2  target:[]  scale:[0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248, 0.408248]
    TASK 'F_qQuaternionNorms/0-#56' (0..89)  type:eq  order:0  target:[]  scale:[3]
    TASK 'F_qZeroVel/1-stick' (29..58)  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_Pose/1-stick' (28..30)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_PoseRel/1-stick-R_endeff' (58..59)  type:eq  order:1  target:[]  scale:[100]
    TASK 'F_LinAngVel/2-stick' (58..60)  type:eq  order:2  target:[]  scale:[1]
    TASK 'F_qZeroVel/1-stick' (59..89)  type:eq  order:1  target:[]  scale:[10]
    TASK 'F_PairCollision/0-R_en

## demo for testing
Calling the rai optimizer as below is just for testing - you're supposed to instead implement your own solvers!

In [11]:
#setup a solver with that problem
solver = ry.NLP_Solver()
solver.setProblem(nlp)
solver.setSolver(ry.NLP_SolverID.augmentedLag)

In [12]:
#solve it
solver.solve(True)

array([ 2.30573125e-04, -2.99986719e-01,  1.07777304e-04, ...,
       -9.96337522e-02, -1.34002916e+00, -1.19075127e-01])

In [13]:
# the trace of queries
solver.getTrace_x()

array([[-9.79108285e-03, -3.26862870e-01,  1.19123239e-02, ...,
         3.46911551e-02, -1.30488362e+00, -1.02992746e-02],
       [-7.95877261e-03, -3.21811814e-01,  9.65355424e-03, ...,
         2.93054177e-02, -1.30406252e+00, -9.86888543e-03],
       [-6.11341050e-03, -3.16739020e-01,  7.38767997e-03, ...,
         2.33304819e-02, -1.30327553e+00, -9.84431256e-03],
       ...,
       [ 2.01811783e-04, -2.99991926e-01,  8.63423209e-05, ...,
        -9.80439892e-02, -1.33937439e+00, -1.18711450e-01],
       [ 2.13447289e-04, -2.99989759e-01,  9.49627579e-05, ...,
        -9.87231451e-02, -1.33965218e+00, -1.18923475e-01],
       [ 2.30573125e-04, -2.99986719e-01,  1.07777304e-04, ...,
        -9.96337522e-02, -1.34002916e+00, -1.19075127e-01]])

In [14]:
#the trace of objectives for each query: the first is total cost, 2nd is ineq, 3rd is eq
solver.getTrace_costs()

array([[3.36357166e+02, 0.00000000e+00, 2.41564632e+03],
       [2.20891307e+02, 0.00000000e+00, 1.96708690e+03],
       [1.29445698e+02, 0.00000000e+00, 1.51810472e+03],
       [6.26455287e+01, 0.00000000e+00, 1.07283507e+03],
       [2.02928313e+01, 0.00000000e+00, 6.27480292e+02],
       [2.46377628e+00, 0.00000000e+00, 2.13646263e+02],
       [6.64943783e-01, 0.00000000e+00, 4.31379676e+01],
       [4.74207320e-01, 0.00000000e+00, 2.07893672e+01],
       [3.76299899e-01, 0.00000000e+00, 2.38540013e+01],
       [4.18671791e-01, 0.00000000e+00, 1.62077992e+01],
       [3.74389640e-01, 0.00000000e+00, 1.86239956e+01],
       [3.93328303e-01, 0.00000000e+00, 1.36230644e+01],
       [3.54560394e-01, 0.00000000e+00, 1.38451897e+01],
       [3.72633012e-01, 0.00000000e+00, 1.19025762e+01],
       [3.57109685e-01, 0.00000000e+00, 9.42771591e+00],
       [3.13876505e-01, 0.00000000e+00, 1.04422850e+01],
       [3.34138938e-01, 0.00000000e+00, 8.39106146e+00],
       [3.22495702e-01, 0.00000

In [ ]:
print(nlp.report(6))

In [14]:
nlp = 0
benchmark = 0
solver = 0